In [10]:
from extractors.pdf_extractor import extract_pdf_as_markdown
from chains.audit_chain import analyze_module
from chains.improvement_chain import get_improvement_chain
from langchain_community.chat_models import ChatOpenAI
import pandas as pd
import os
from dotenv import load_dotenv
import asyncio
import os
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

In [2]:
load_dotenv()
# print(os.getenv("OPENAI_API_KEY"))

file_path = "./files/Business Continuity Plan E-WISE [Restricted].docx.pdf"
controls = pd.read_json("ISO_27001_2022_Controls_List.json")


In [3]:
controls

,Section,Control ID,Control Title
0,Clause 4,4.1,Understanding the organization and its context
1,Clause 4,4.2,Understanding the needs and expectations of in...
2,Clause 4,4.3,Determining the scope of the ISMS
3,Clause 4,4.4,Information security management system
4,Clause 5,5.1,Leadership and commitment
...,...,...,...
110,A.8 (Technological Controls),A.8.30,Control title placeholder for A.8.30
111,A.8 (Technological Controls),A.8.31,Control title placeholder for A.8.31
112,A.8 (Technological Controls),A.8.32,Control title placeholder for A.8.32
113,A.8 (Technological Controls),A.8.33,Control title placeholder for A.8.33


In [4]:
n = len(controls)
part_size = n // 3

df1 = controls.iloc[:part_size]
df2 = controls.iloc[part_size:2*part_size]
df3 = controls.iloc[2*part_size:]

# Optional: reset index
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)
df3 = df3.reset_index(drop=True)

In [5]:
text = extract_pdf_as_markdown(file_path)


✔️ Tesseract detected: tesseract 4.1.1


In [ ]:

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3, api_key=os.getenv("OPENAI_API_KEY"))
clauses = [df1,df2,df3]
audits = []
results = []
for clause in clauses:
    audit_chain = analyze_module(llm)
    results.append(audit_chain.run(text=text, control_json=clause.to_json(orient='records', indent=2)))



✔️ Tesseract detected: tesseract 4.1.1


/tmp/ipykernel_13772/3417714339.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3, api_key=os.getenv("OPENAI_API_KEY"))
/home/sherry/DocAnalyzer/Document_Analyzer_Streamlit_App/chains/audit_chain.py:40: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(llm=llm, prompt=prompt)
/tmp/ipykernel_13772/3417714339.py:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results.append(audit_chain.run(text=te

In [ ]:
from langchain.callbacks import get_openai_callback
import nest_asyncio
import sys
nest_asyncio.apply()

text = extract_pdf_as_markdown(file_path)

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3,streaming=False, api_key=os.getenv("OPENAI_API_KEY"))
clauses = [df1, df2, df3]
results = []
total_tokens = 0
total_cost = 0.0

async def process_clause_async(clause, text, llm):
    def sync_callback_wrapper():
        with get_openai_callback() as cb:
            audit_chain = analyze_module(llm)
            result = audit_chain.run(
                text=text,
                control_json=clause.to_json(orient='records', indent=2)
            )
            return {
                "result": result,
                "tokens": cb.total_tokens,
                "cost": cb.total_cost,
                "controls": len(clause)
            }

    return await asyncio.to_thread(sync_callback_wrapper)

async def run_all_clauses():
    results = []
    total_tokens = 0
    total_cost = 0.0

    tasks = [process_clause_async(clause, text, llm) for clause in clauses]
    clause_results = await asyncio.gather(*tasks)

    for data in clause_results:
        results.append(data["result"])
        total_tokens += data["tokens"]
        total_cost += data["cost"]
        print(f"🔎 Clause Processed: {data['controls']} controls")
        print(f"🧠 Tokens used: {data['tokens']}")
        print(f"💵 Cost: ${data['cost']:.6f}")

    print("\n✅ All clauses processed (async).")
    print(f"🔢 Total tokens used: {total_tokens}")
    print(f"💰 Total cost: ${total_cost:.6f}")

    return results

if __name__ == "__main__":
    if sys.platform == "win32":
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    final_results = await run_all_clauses() if 'google.colab' in sys.modules or 'IPython' in sys.modules else asyncio.run(run_all_clauses())


✔️ Tesseract detected: tesseract 4.1.1


RuntimeError: asyncio.run() cannot be called from a running event loop

In [7]:
results

['```json\n[\n  {\n    "Clause": "4",\n    "Title": "Context of the organization",\n    "Compliance": "❌",\n    "Gaps Identified": "No information provided about understanding the organization and its context.",\n    "Recommended Action": "Include a detailed analysis of the organization\'s internal and external issues relevant to the ISMS."\n  },\n  {\n    "Clause": "4",\n    "Title": "Understanding the needs and expectations of interested parties",\n    "Compliance": "❌",\n    "Gaps Identified": "No identification or consideration of interested parties and their requirements.",\n    "Recommended Action": "Document the interested parties relevant to the ISMS and their needs and expectations."\n  },\n  {\n    "Clause": "4",\n    "Title": "Determining the scope of the ISMS",\n    "Compliance": "❌",\n    "Gaps Identified": "No defined scope for the ISMS is mentioned.",\n    "Recommended Action": "Define and document the scope of the ISMS including boundaries and applicability."\n  },\n  {

In [6]:
import json
import pandas as pd

# Parse each JSON string and collect all items into a single list
all_items = []
for js in results:
    try:
        # Remove optional code formatting like ```json and ```
        clean_js = js.strip().removeprefix("```json").removesuffix("```").strip()
        all_items.extend(json.loads(clean_js))
    except json.JSONDecodeError as e:
        print("Failed to parse:", js)
        raise e

# Convert to DataFrame
resutls_df = pd.DataFrame(all_items)



In [7]:
resutls_df

,Section,Control Id,Control Title,Compliance,Gaps Identified,Recommended Action
0,4,4.1,Understanding the organization and its context,🟡,The document references the scope of the ISMS ...,Include a detailed analysis of the organizatio...
1,4,4.2,Understanding the needs and expectations of in...,🟡,The document identifies some interested partie...,Document and analyze all relevant interested p...
2,4,4.3,Determining the scope of the ISMS,🟡,The document mentions the scope applies to all...,Develop and document a formal ISMS scope state...
3,4,4.4,Information security management system,❌,The document does not describe the establishme...,"Include a description of the ISMS framework, p..."
4,5,5.1,Leadership and commitment,🟡,Leadership roles and responsibilities are defi...,"Document leadership commitment to the ISMS, in..."
...,...,...,...,...,...,...
110,A.8 (Technological Controls),A.8.30,Control title placeholder for A.8.30,❌,A.8.30 controls not addressed.,Document and implement A.8.30 controls.
111,A.8 (Technological Controls),A.8.31,Control title placeholder for A.8.31,❌,No evidence of A.8.31 controls.,Add A.8.31 controls.
112,A.8 (Technological Controls),A.8.32,Control title placeholder for A.8.32,❌,A.8.32 controls missing.,Implement A.8.32 controls.
113,A.8 (Technological Controls),A.8.33,Control title placeholder for A.8.33,❌,No mention of A.8.33 controls.,Add A.8.33 controls.


In [8]:
resutls_df.to_excel("result.xlsx")

In [8]:
with open("output.md", "w", encoding="utf-8") as f:
    f.write(text)

print("✅ PDF content extracted and saved as 'output.md'")

✅ PDF content extracted and saved as 'output.md'
